In [54]:
import nltk
nltk.download('punkt')
# nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import spacy


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [1]:
import re
import math
import pandas as pd

import jieba
from collections import OrderedDict

from pprint import pprint

In [2]:
jobs7 = pd.read_csv('job7_4groups.csv', encoding='utf-8-sig')

In [3]:
text = pd.DataFrame({
    'category': jobs7["職位_"],
    'content': list(jobs7["工作內容"] + ' ' + jobs7["其他條件"] + ' ' + jobs7["工作技能"])
})

In [4]:
text.head()

,category,content
0,職位 IV,1.系統功能需求分析設計。\n2.系統功能規劃與架構建構佈署。\n3.具出色的溝通能力與組織...
1,職位 IV,1、負責NBS升級案之專案執行之文件與進度管理2、監督工作成員的各項工作進度回報3、與客戶進...
2,職位 IV,VICI Holdings 威旭資訊是一間專注於高頻、造市及套利交易的公司，我們進行量化研究...
3,職位 IV,1. 熟悉反洗錢架構2. 熟悉SAS AML系統3. 有帶領團隊經驗4. 善溝通、有技術程式...
4,職位 IV,洞察和量化系統流程瓶頸，關注服務穩定性 發掘系統 performance 和 secur...


In [5]:
len(text)

57127

In [6]:
text = text.dropna().reset_index()
len(text)

56392

In [7]:
text = text.groupby(['category'])['content'].apply(lambda x: ' '.join(x)).reset_index()
text

,category,content
0,職位 I,1.對於微控制器程式設計有興趣的人員※工作內容：1. 量測、控制、通訊及人機介面等韌體單元設...
1,職位 II,1.負責網路安控軟體系統開發、系統測試2.熟PKI、專案管理、Java、Asp、Jsp、Ja...
2,職位 III,1.IC驗證 \n2.驗證系統開發 \n3.客戶design in問題解決 \n4.FPGA...
3,職位 IV,1.系統功能需求分析設計。\n2.系統功能規劃與架構建構佈署。\n3.具出色的溝通能力與組織...


In [8]:
#拔符號

In [9]:
r_1 = r'https?://[a-zA-Z0-9./_%-@=]+|[a-zA-Z0-9]+@[a-zA-Z0-9.]+|@[a-zA-Z0-9.]+|\d\.|\(\d\)'
r_2 = r'，|,|。|…|！|!|？|\?|、|､|；|【|】|〖|〗|「|」|｢|:|：|;|；|＆|&|\(|（|\)|）|{|}|・|．|<|>|｜|_|-|—|–|╴|￣|\[|]|［|］|\*|%|~|～|\'|\"|{|}|｛|｝|\^|⌃|\$|\||●|•|·|°|★|✦|❖|￭|■|◆|※|✓|√|⇧|⌘|＝|=|@|ž|\r'

In [10]:
content1 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

for i in range(len(text)):
    c00 = re.sub(r_1, ' ', text.loc[i, 'content'])
    c01 = re.sub(r_2, ' ', c00)
    c02 = re.sub(r'^\s+', '', c01)
    content1.loc[i, 'content'] = c02

In [11]:
content1

,category,content
0,職位 I,對於微控制器程式設計有興趣的人員 工作內容 量測 控制 通訊及人機介面等韌體單元設計撰寫...
1,職位 II,負責網路安控軟體系統開發 系統測試 熟PKI 專案管理 Java Asp Jsp Java ...
2,職位 III,IC驗證 \n 驗證系統開發 \n 客戶design in問題解決 \n FPGA驗證 熟電...
3,職位 IV,系統功能需求分析設計 \n 系統功能規劃與架構建構佈署 \n 具出色的溝通能力與組織協調能力...


In [12]:
#拔停頓詞

In [13]:
stop = r'的|你|妳|我|他|們|您|這個|那個|與|和|或|及|之|等|都|不只|是|以及|每次|提供|例如|並|者|又|至|最|於|需|需要|而且|且|目前|尋找|想找|尤佳|更|佳|希望|職缺|公司|工作|內容|職務|面試|面談|人才|履歷|職位|包含|一間|具備|熟悉|熟練|成立|主要|單位|眾多|負責|歡迎|加入|至少|使用|相關|擔任|回覆|對於|協助|辦理|事務|執行|關於|以下|其他|致力|身為|簡介|企業|另行|懂|非常|謝謝|帶來|若|如果|如下|未填寫|不拘'
stop_eng = r'\ba\b|\bi\b|\byou\b|\bwe\b|\bis\b|\bare\b|\bwas\b|\bwere\b|\band\b|\bthe\b|\bin\b|\bfor\b|\bon\b|\bto\b|\bat\b|\bof\b|\bfujitsu\b'
stop_jap = r'[ぁ-んァ-ンｧ-ﾝﾞﾟ｡]'

In [14]:
content2 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

for i in range(len(content1)):
    c10 = re.sub(stop, ' ', content1.loc[i, 'content'])
    c11 = c10.lower()
    c12 = re.sub(stop_eng, ' ', c11)
    c13 = re.sub(stop_jap, ' ', c12)
    c14 = re.sub(r'\s+', ' ', c13)
    content2.loc[i, 'content'] = c14

In [15]:
content2

,category,content
0,職位 I,微控制器程式設計有興趣 人員 量測 控制 通訊 人機介面 韌體單元設計撰寫 驗證 配合韌體...
1,職位 II,網路安控軟體系統開發 系統測試 熟pki 專案管理 java asp jsp java s...
2,職位 III,ic驗證 驗證系統開發 客戶design in問題解決 fpga驗證 熟電子電路原理 任一技...
3,職位 IV,系統功能 求分析設計 系統功能規劃 架構建構佈署 具出色 溝通能力 組織協調能力 邏輯分析能...


In [16]:
#切關鍵字

In [17]:
# jieba.load_userdict(r'C:\Users\USER\Final_Project\dic\trad_dict.txt')
# jieba.load_userdict(r'C:\Users\USER\Final_Project\dic\wiki_lower.txt')
# jieba.load_userdict(r'C:\Users\USER\Final_Project\dic\ee_lower.txt')
# jieba.load_userdict(r'C:\Users\USER\Final_Project\dic\computer_lower.txt')
# jieba.load_userdict(r'C:\Users\USER\Final_Project\dic\statistics_lower.txt')
# jieba.load_userdict(r'C:\Users\USER\Final_Project\dic\library_lower.txt')

In [18]:
# # jieba 加字典
# # 新增詞庫
# with open(r'C:\Users\USER\Final_Project\job7_dic\trad_dict.txt', 'r', encoding='utf-8') as infile:
#     d = infile.read()
# for word in d:
#     jieba.add_word(word)

# path_dic = [r'C:\Users\USER\Final_Project\job7_dic\wiki_lower.txt',
#             r'C:\Users\USER\Final_Project\job7_dic\ee_lower.txt',
#             r'C:\Users\USER\Final_Project\job7_dic\computer_lower.txt',
#             r'C:\Users\USER\Final_Project\job7_dic\statistics_lower.txt',
#             r'C:\Users\USER\Final_Project\job7_dic\library_lower.txt']
# for dic in path_dic:
#     with open(dic, 'r', encoding='utf-8') as infile:
#         d = infile.read()
#     for word in d:
#         jieba.add_word(word+' 1000')

In [19]:
jieba.add_word('asp.net')
jieba.add_word('區塊鍊')
jieba.add_word('區塊鏈')
jieba.add_word('加分項')
jieba.add_word('加分')
jieba.add_word('sas')
jieba.add_word('aml')
jieba.add_word('es6+')
jieba.add_word('信託')
jieba.add_word('大數據')
jieba.add_word('撰寫')
jieba.add_word('身體')
jieba.add_word('公佈')
jieba.add_word('方向')
jieba.add_word('吸引力')
jieba.add_word('為什麼')
jieba.add_word('製作')
jieba.add_word('mysql')
jieba.add_word('mssql')
jieba.add_word('sqlite')
jieba.add_word('rfp')
jieba.add_word('新創')
jieba.add_word('工程師')
jieba.add_word('帶領')
jieba.add_word('佈署')
jieba.add_word('製作')
jieba.add_word('追蹤')
jieba.add_word('開發')
jieba.add_word('經驗')
jieba.add_word('優化')
jieba.add_word('作業')
jieba.add_word('透過')
jieba.add_word('程式碼')
jieba.add_word('回應')
jieba.add_word('不確定')
jieba.add_word('委託')
jieba.add_word('接下來')
jieba.add_word('壓力測試')
jieba.add_word('沒有')
jieba.add_word('比特幣')
jieba.add_word('彼特幣')
jieba.add_word('最大值')
jieba.add_word('最小值')
jieba.add_word('訓練集')
jieba.add_word('邏輯式迴歸')
jieba.add_word('邏輯式回歸')
jieba.add_word('羅吉斯迴歸')
jieba.add_word('羅吉斯回歸')
jieba.add_word('迴歸')
jieba.add_word('線性迴歸')
jieba.add_word('線性回歸')
jieba.add_word('特徵')
jieba.add_word('程式碼')
jieba.add_word('物件導向')
jieba.add_word('前端')
jieba.add_word('後端')
jieba.add_word('全端')
jieba.add_word('前後端')
jieba.add_word('工程師')
jieba.add_word('分類器')
jieba.add_word('機器學習')
jieba.add_word('深度學習')
jieba.add_word('演算法')
jieba.add_word('監督式')
jieba.add_word('機率')
jieba.add_word('人工智慧')
jieba.add_word('決策樹')
jieba.add_word('數據分析')
jieba.add_word('資料探勘')
jieba.add_word('正規化')
jieba.add_word('過度配適')
jieba.add_word('特徵工程')
jieba.add_word('特徵選取')
jieba.add_word('svm')
jieba.add_word('random forest')
jieba.add_word('隨機森林')
jieba.add_word('sklearn')
jieba.add_word('矩陣')
jieba.add_word('自然語言處理')
jieba.add_word('logistic regression')
jieba.add_word('linear regression')
jieba.add_word('k means')
jieba.add_word('主成分分析')
jieba.add_word('統計')
jieba.add_word('maximum likelihood estimation')
jieba.add_word('reinforcement learning')
jieba.add_word('decision tree')
jieba.add_word('id3')
jieba.add_word('naive bayes')
jieba.add_word('polynomial regression')
jieba.add_word('lasso regression')
jieba.add_word('ridge regression')
jieba.add_word('elasticnet regression')
jieba.add_word('網頁設計')
jieba.add_word('專案管理')
jieba.add_word('梯度下降法')
jieba.add_word('資安')
jieba.add_word('word embedding')
jieba.add_word('網路爬蟲')
jieba.add_word('資工')
jieba.add_word('叢集')
jieba.add_word('迴歸樹')
jieba.add_word('回歸樹')
jieba.add_word('分類樹')
jieba.add_word('管理員')
jieba.add_word('系統')
jieba.add_word('安裝')
jieba.add_word('邏輯斯迴歸')
jieba.add_word('邏輯斯回歸')
jieba.add_word('裝置')
jieba.add_word('多人')
jieba.add_word('多工')
jieba.add_word('人員')
jieba.add_word('系統化')
jieba.add_word('系統化架構')
jieba.add_word('c語言')
jieba.add_word('諮詢')
jieba.add_word('複製')
jieba.add_word('防火牆')
jieba.add_word('資料庫')
jieba.add_word('vue.js')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\student\AppData\Local\Temp\jieba.cache
Loading model cost 0.477 seconds.
Prefix dict has been built successfully.


In [20]:
content3 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

In [21]:
#精確模式（Default Mode）
seg_list = []
for i in range(len(content2)):
    seg_list = jieba.cut(content2.loc[i, 'content'], cut_all=False) #預設是精確模式　seg_list = jieba.cut(test)
    content3.loc[i, 'content'] = " ".join(seg_list)

In [22]:
content3

,category,content
0,職位 I,微控制器 程式 設計 有 興趣 人員 量測 控制 通訊 人機 介面 ...
1,職位 II,網路安控 軟體 系統 開發 系統 測試 熟 pki 專案管理 java ...
2,職位 III,ic 驗證 驗證 系統 開發 客戶 design in 問題 解決 fpga...
3,職位 IV,系統 功能 求 分析 設計 系統 功能 規劃 架構 建構 佈署 具 出色 ...


In [23]:
# content3.loc[0, 'content'][30000:40000]

In [24]:
# print("搜索引擎模式（For Search）")
# for i in range(100, 200):
#     print(i)
#     seg_list = jieba.cut_for_search(content2[i])
#     print("/ ".join(seg_list))

In [25]:
r_3 = r'\b\d+\b'

In [26]:
content4 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

for j in range(len(content3)):
    c20 = re.sub(r_3, ' ', content3.loc[j, 'content'])
    c21 = c20.split()
    c_list = []
    i = len(c21)-1
    while i>=0:
        if 'c' not in c21[i]:
            c21[i] = c21[i].replace("#", "")
            c21[i] = c21[i].replace("+", "")
        if len(c21[i]) <= 1:
            i-=1
            continue
        if c21[i] == '.' or c21[i] == '..'or c21[i] == '...' or c21[i] == '....' or c21[i] == '.....' \
        or c21[i] == '......'or c21[i] == '.......'or c21[i] == '........'or c21[i] == '.........'or c21[i] == '..........'\
        or c21[i] == '/' or c21[i] == '//' or c21[i] == '\\' or c21[i] == '\\\\'\
        or (not c21[i].isprintable()):
            i -= 1
            continue
        if c21[i] == 'co' and c21[i-1] == '/' and c21[i-2] == 'fi':
            if c21[i-3] == 'hana':
                c_list.append('s/4 hana fi/co')
                i -= 7
                continue
            elif c21[i-3] == '4' and c21[i-4] == '\/' and c21[i-5] == 's':
                c_list.append('s/4 fi/co')
                i -= 6
                continue
        if c21[i] == 'hana':
            if c21[i-1] == '4' and c21[i-2] == '/' and c21[i-3] == 's':
                i -= 4
                continue
        c_list.append(c21[i])
        i -= 1
    content4.loc[j, 'content'] = c_list[::-1]

In [27]:
content4

,category,content
0,職位 I,"[微控制器, 程式, 設計, 興趣, 人員, 量測, 控制, 通訊, 人機, 介面, 韌體,..."
1,職位 II,"[網路安控, 軟體, 系統, 開發, 系統, 測試, pki, 專案管理, java, as..."
2,職位 III,"[ic, 驗證, 驗證, 系統, 開發, 客戶, design, in, 問題, 解決, f..."
3,職位 IV,"[系統, 功能, 分析, 設計, 系統, 功能, 規劃, 架構, 建構, 佈署, 出色, 溝..."


In [68]:
#tf idf

In [70]:
uniqueWords = set()
for i in range(len(content4)):
    a = content4.loc[i,"content"]
    uniqueWords = uniqueWords.union(set(a))

In [71]:
len(uniqueWords)

61400

In [72]:
# 職缺 I 的 bag of word 統計
numOfWords = dict.fromkeys(uniqueWords, 0)

# 初始資料表
df_por = pd.DataFrame(
{'por':numOfWords}
)


for i in range(len(content4)):
    a = content4.loc[i,"content"]
    for word in a:
        numOfWords[word] += 1

    # 第i職缺
    df = pd.DataFrame({i:numOfWords}) 
    df_por = pd.concat([df_por, df], axis = 1)

    # 初始化
    numOfWords = dict.fromkeys(uniqueWords, 0)


In [73]:
df_por.drop('por', axis=1, inplace=True)
df_por

,0,1,2,3
...............................................,0,2,1,0
.........................................................................,0,3,4,0
000jpy,0,3,0,1
000senior,0,3,0,1
00am,0,0,3,0
...,...,...,...,...
龐大,0,52,6,10
龐大資源,0,4,0,0
龐雜,0,9,2,0
龜山,0,12,16,1


In [74]:
df_por

,0,1,2,3
...............................................,0,2,1,0
.........................................................................,0,3,4,0
000jpy,0,3,0,1
000senior,0,3,0,1
00am,0,0,3,0
...,...,...,...,...
龐大,0,52,6,10
龐大資源,0,4,0,0
龐雜,0,9,2,0
龜山,0,12,16,1


In [75]:
df_por[0].to_dict()

{'...............................................': 0,
 '.........................................................................': 0,
 '000jpy': 0,
 '000senior': 0,
 '00am': 0,
 '00am10': 0,
 '00am5': 0,
 '00pm': 0,
 '02years': 0,
 '045mw': 0,
 '06y': 0,
 '0m': 0,
 '10012la': 0,
 '100b': 0,
 '100e': 0,
 '100g': 0,
 '100gb': 0,
 '100k': 0,
 '100k150k': 0,
 '100m': 0,
 '100tb': 0,
 '100w': 0,
 '105k': 0,
 '10g': 0,
 '10gbps': 0,
 '10gepon': 0,
 '10goracle': 0,
 '10gpon': 0,
 '10m': 0,
 '10microsoft': 0,
 '10nas': 0,
 '10office': 0,
 '10oracle': 0,
 '10pon': 0,
 '10tb': 0,
 '10x': 0,
 '10years': 0,
 '110c': 0,
 '1110627e': 0,
 '112g': 0,
 '112gbps': 0,
 '11a': 0,
 '11ac': 0,
 '11ax': 0,
 '11b': 0,
 '11be': 0,
 '11c': 0,
 '11g': 0,
 '11k': 0,
 '11n': 0,
 '11pm': 0,
 '127m': 0,
 '12c': 1,
 '12cm': 0,
 '12f.': 0,
 '12factor': 0,
 '12g': 0,
 '12h': 0,
 '12hr': 0,
 '12k': 0,
 '12pm9pm': 0,
 '12th': 0,
 '12v': 0,
 '13th': 0,
 '1400k': 0,
 '144hz': 0,
 '14f': 0,
 '150mw': 0,
 '150x70': 0,
 '15

In [76]:
# TF

def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [77]:
numOfWords = dict.fromkeys(uniqueWords, 0)

df_TF = pd.DataFrame(
{'TF_por':numOfWords}
)

for i in range(len(content4)):
    a = content4.loc[i,'content']

    # 第i職缺
    df = pd.DataFrame({i:computeTF(df_por[i].to_dict(),a)}) 
    df_TF = pd.concat([df_TF, df], axis = 1)

    # 初始化
    df = dict.fromkeys(uniqueWords, 0)
    


df_TF

,TF_por,0,1,2,3
...............................................,0,0.0,7.983866e-07,4.675200e-07,0.000000
.........................................................................,0,0.0,1.197580e-06,1.870080e-06,0.000000
000jpy,0,0.0,1.197580e-06,0.000000e+00,0.000002
000senior,0,0.0,1.197580e-06,0.000000e+00,0.000002
00am,0,0.0,0.000000e+00,1.402560e-06,0.000000
...,...,...,...,...,...
龐大,0,0.0,2.075805e-05,2.805120e-06,0.000020
龐大資源,0,0.0,1.596773e-06,0.000000e+00,0.000000
龐雜,0,0.0,3.592740e-06,9.350400e-07,0.000000
龜山,0,0.0,4.790320e-06,7.480320e-06,0.000002


In [83]:
df_TF.loc['料庫',1]

0.0

In [85]:
# df_TF.drop('TF_por', axis=1, inplace=True)
df_TF

,0,1,2,3
...............................................,0.0,7.983866e-07,4.675200e-07,0.000000
.........................................................................,0.0,1.197580e-06,1.870080e-06,0.000000
000jpy,0.0,1.197580e-06,0.000000e+00,0.000002
000senior,0.0,1.197580e-06,0.000000e+00,0.000002
00am,0.0,0.000000e+00,1.402560e-06,0.000000
...,...,...,...,...
龐大,0.0,2.075805e-05,2.805120e-06,0.000020
龐大資源,0.0,1.596773e-06,0.000000e+00,0.000000
龐雜,0.0,3.592740e-06,9.350400e-07,0.000000
龜山,0.0,4.790320e-06,7.480320e-06,0.000002


In [86]:
# IDF
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    return idfDict

In [87]:
df_por_list = [] 

for i in range(len(content4)):
    df_por_list.append(df_por[i].to_dict())


idfs = computeIDF(df_por_list)

In [88]:
df_idfs = pd.DataFrame({'idf':idfs},)
df_idfs.sort_values(by=["idf"],ascending=False)

,idf
龜山區,0.60206
榮膺,0.60206
業頂,0.60206
systemincrease,0.60206
業預測,0.60206
...,...
service,0.00000
networking,0.00000
完成,0.00000
薪資以,0.00000


In [90]:
df_idfs.loc['資料庫',:]

idf    0.124939
Name: 資料庫, dtype: float64

In [91]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [92]:
df_por

,0,1,2,3
...............................................,0,2,1,0
.........................................................................,0,3,4,0
000jpy,0,3,0,1
000senior,0,3,0,1
00am,0,0,3,0
...,...,...,...,...
龐大,0,52,6,10
龐大資源,0,4,0,0
龐雜,0,9,2,0
龜山,0,12,16,1


In [93]:
tfidf_list = []
for i in range(len(df_por.columns)):
    tfidf = computeTFIDF(df_TF[i].to_dict(), idfs)
    tfidf_list.append(tfidf)

df_tfidf = pd.DataFrame(tfidf_list).T

df_tfidf

,0,1,2,3
...............................................,0.0,2.403383e-07,1.407375e-07,0.000000e+00
.........................................................................,0.0,3.605075e-07,5.629502e-07,0.000000e+00
000jpy,0.0,3.605075e-07,0.000000e+00,6.026482e-07
000senior,0.0,3.605075e-07,0.000000e+00,6.026482e-07
00am,0.0,0.000000e+00,8.444252e-07,0.000000e+00
...,...,...,...,...
龐大,0.0,2.593485e-06,3.504681e-07,2.501216e-06
龐大資源,0.0,9.613533e-07,0.000000e+00,0.000000e+00
龐雜,0.0,1.081522e-06,2.814751e-07,0.000000e+00
龜山,0.0,5.984965e-07,9.345817e-07,2.501216e-07


In [97]:
df_por

# df_por[0].to_dict()

df_tfidf.sort_values([3], ascending=False)[3].head(100).to_dict()



{'資安': 0.0007581185450194532,
 '資料庫': 0.00040194539816768764,
 'app': 0.0003171541785168811,
 '操作': 0.00029139165455218177,
 'web': 0.00022410894633369518,
 'sql': 0.00018508997799881075,
 '證照': 0.0001730841415880771,
 '網站': 0.00016207879154490453,
 '維運': 0.00016082818358545314,
 'ios': 0.00015432502219630572,
 '設定': 0.00014356979374502345,
 'aws': 0.0001383172403153275,
 '雲端': 0.0001383172403153275,
 'mes': 0.00013806711872343721,
 '事項': 0.0001348155380288635,
 '後端': 0.00012981310619105782,
 '檢測': 0.00012055860729111726,
 'net': 0.00011130410839117673,
 '串接': 0.00010955325724794473,
 '金融': 0.00010830264928849331,
 'erp': 0.00010305009585879732,
 '說明': 0.00010054887993989448,
 '業務': 9.929827198044306e-05,
 '產業': 9.404571855074709e-05,
 'iso': 9.304523218318594e-05,
 '防火牆': 9.154450263184423e-05,
 'framework': 8.929340830483166e-05,
 '數據': 8.87931651210511e-05,
 '簡報': 8.704231397781911e-05,
 '認證': 8.629194920214826e-05,
 'azure': 8.529146283458711e-05,
 'mysql': 8.454109805891625e-05,
 